<a href="https://colab.research.google.com/github/masoudndata/Data-Science-training-projects/blob/main/Time_Forcasting_for_Prediction_the_Global_Temerature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from fbprophet import Prophet
from plotly.subplots import make_subplots

In [2]:
url = 'https://github.com/masoudndata/Data-Science-projects/blob/main/Environment_Temperature_change_E_All_Data_NOFLAG.csv?raw=true'
data = pd.read_csv(url, encoding = 'unicode_escape', engine ='python')

In [3]:
data

,Area Code,Area,Months Code,Months,Element Code,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,7001,January,7271,Temperature change,°C,0.777,0.062,2.744,-5.232,1.868,3.629,-1.432,0.389,-2.298,0.804,-1.487,-1.305,-2.951,-1.184,-0.490,2.409,-3.014,-0.663,1.141,-0.393,1.724,0.678,0.524,-0.058,0.435,0.332,2.655,1.150,-1.108,0.634,0.018,0.582,-0.821,1.087,1.297,-0.718,1.426,0.950,0.859,1.565,-0.603,1.606,2.479,2.707,0.109,-1.606,0.431,-5.553,1.518,3.601,1.179,-0.583,1.233,1.755,1.943,3.416,1.201,1.996,2.951
1,2,Afghanistan,7001,January,6078,Standard Deviation,°C,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950
2,2,Afghanistan,7002,February,7271,Temperature change,°C,-1.743,2.465,3.919,-0.202,-0.096,3.397,0.296,-2.055,-3.167,1.809,0.816,-7.722,1.838,-3.706,-1.239,-1.620,-0.156,-0.369,1.072,-1.222,1.088,-2.101,0.460,-4.321,2.467,-0.286,1.409,0.170,-2.890,-0.310,-1.373,-0.120,1.414,-1.412,-0.149,0.870,0.043,-0.540,3.222,-0.901,0.707,0.985,1.816,2.871,-1.506,4.725,1.645,-2.332,2.494,1.212,0.321,-3.201,1.494,-3.187,2.699,2.251,-0.323,2.705,0.086
3,2,Afghanistan,7002,February,6078,Standard Deviation,°C,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597
4,2,Afghanistan,7003,March,7271,Temperature change,°C,0.516,1.336,0.403,1.659,-0.909,-0.069,-0.759,0.496,2.481,-0.915,1.658,-1.784,-0.473,1.001,-0.585,-2.817,3.377,-1.536,-1.420,-0.628,1.166,-1.781,-2.406,1.761,-0.160,-3.087,1.330,-0.056,0.065,-1.000,-0.901,-2.220,-1.449,1.312,-1.451,-0.336,-0.005,-0.576,-0.217,-0.267,1.229,1.949,-0.158,2.753,1.663,1.847,-0.201,4.172,2.362,3.390,0.748,-0.527,2.246,-0.076,-0.497,2.296,0.834,4.418,0.234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9651,5873,OECD,7018,JunJulAug,6078,Standard Deviation,°C,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247
9652,5873,OECD,7019,SepOctNov,7271,Temperature change,°C,0.036,0.461,0.665,-0.157,-0.203,-0.295,0.321,0.220,-0.027,-0.146,-0.004,-0.636,0.132,-0.544,-0.124,-0.537,0.374,-0.441,0.525,0.317,0.575,-0.101,0.673,-0.290,-0.528,-0.502,0.672,0.284,0.012,0.209,0.043,-0.385,-0.388,0.623,0.285,-0.202,0.576,1.285,0.834,0.539,0.831,0.735,0.904,0.748,1.230,0.972,0.909,0.785,1.093,0.958,1.106,0.885,1.041,0.999,1.670,1.535,1.194,0.581,1.233
9653,5873,OECD,7019,SepOctNov,6078,Standard Deviation,°C,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.37

In [4]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Remove the standard deviation examples
    df = df.query("Element == 'Temperature change'")
    
    # Group countries and take their means
    df = df.groupby('Area').mean()
    
    # Use only time series data
    df = df.loc[:, 'Y1961':]
    
    # Get global mean temperature change
    df = pd.DataFrame(df.mean()).reset_index(drop=False)
    
    # Rename columns
    df.columns=['ds', 'y']
    
    # Fix year column
    df['ds'] = df['ds'].apply(lambda x: x[1:]).astype(np.int)
    
    return df

In [5]:
time_series = preprocess_inputs(data)

In [6]:
time_train = time_series.iloc[:44, :].copy()
time_test = time_series.iloc[44:, :].copy()

In [7]:
indata_model = Prophet()
indata_model.fit(time_train)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [8]:
indata_forecast = indata_model.predict(time_test).loc[:, ['ds', 'yhat']]
indata_forecast['ds'] = indata_forecast['ds'].apply(lambda x: x.year)
indata_forecast

,ds,yhat
0,2005,0.811445
1,2006,0.809412
2,2007,0.796941
3,2008,0.774077
4,2009,0.901351
5,2010,0.899318
6,2011,0.886848
7,2012,0.863983
8,2013,0.991258
9,2014,0.989224


In [9]:
subfig = make_subplots(specs=[[{"secondary_y": True}]])
indata_result_df = pd.concat([time_series, indata_forecast], axis=0)

fig = px.line(
    indata_result_df,
    x='ds',
    y='y'
  ) 
fig2 =  px.line(
    indata_result_df,
    x='ds',
    y='yhat'
  ) 


subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="year"
subfig.layout.yaxis.title="change temperature"

subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
subfig.show()

In [10]:
future_df = pd.DataFrame(np.arange(2020, 2046), columns=['ds'])
future_df

,ds
0,2020
1,2021
2,2022
3,2023
4,2024
5,2025
6,2026
7,2027
8,2028
9,2029


In [11]:
outdata_model = Prophet()
outdata_model.fit(time_series)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
outdata_forecast = outdata_model.predict(future_df).loc[:, ['ds', 'yhat']]
outdata_forecast['ds'] = outdata_forecast['ds'].apply(lambda x: x.year)
outdata_forecast

,ds,yhat
0,2020,1.216573
1,2021,1.328136
2,2022,1.354637
3,2023,1.354043
4,2024,1.326387
5,2025,1.437950
6,2026,1.464451
7,2027,1.463857
8,2028,1.436201
9,2029,1.547764


In [13]:
subfig = make_subplots(specs=[[{"secondary_y": True}]])
outdata_result_df = pd.concat([time_series, outdata_forecast], axis=0)

fig = px.line(
    outdata_result_df,
    x='ds',
    y='y'
)
fig2 =  px.line(
    outdata_result_df,
    x='ds',
    y='yhat'
  ) 


subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="year"
subfig.layout.yaxis.title="change temperature"
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))

subfig.show()